# 목표
1. 중복 이벤트 삭제 함수 만들기
2. Replacement 함수 만들기
3. Reintubation 함수 만들기

## 1. 삽관 및 발관 중복 이벤트 삭제하기

### Workflow 1-1
* 기존에 있는 intubation1, extubation1 dataframe으로 함수 작성

* 중복 제거 후, time_diff를 구하기 위해 dtype 변경

* loc를 원활히 활용하기 위해 index 초기화 진행

* 첫 번째 행 (0번째 행)은 겹칠 일이 없기에 not_duplicated로 지정

* for문을 활용하여 index를 순차적 조회
    * prev_row, current_row의 subject_id, hadm_id가 같다면 같은 환자 및 시기로 판단
    * time_difference = int_time or ext_time의 current_row - prev_row
    * time_difference <= 1 이라면, duplicate로 지정
        * 추후 확인을 위해 time_difference <= 1 인 row와 prev_row 두개 duplicate로 지정

In [1]:
import time
import pandas as pd
intubation1 = pd.read_csv('./Data/intubation1.csv')
extubation1 = pd.read_csv('./Data/extubation1.csv')

In [4]:
def time_duplicate_event(Original_dataframe, time_col):
    
    # DataFrame 중복 제거
    origin_df = Original_dataframe.drop_duplicates()

    # time delta를 구하기 위해 dtype 변경
    origin_df.loc[:,time_col] = pd.to_datetime(origin_df[time_col])

    # subject_id, hadm_id, int_time(or ext_time) 순으로 정렬
    sorted_df = origin_df.sort_values(by=["subject_id", "hadm_id", time_col], ascending=True)

    # index 초기화 (loc 함수를 원활히 사용하기 위함)
    sorted_df = sorted_df.reset_index(drop=True)
    
    length_df = len(sorted_df)

    # 첫 번째 행 (0번째 행)은 겹칠일이 없기에 not_duplicated 지정
    sorted_df.loc[0, "dup"] = "not_duplicated"

    # dataframe의 길이를 가지고 for문 시작.
    for idx in range(1, (length_df)):
        
        # 현재 행과 이전 행의 subject_id가 같고 hadm_id가 같다는 조건을 동시에 만족한다면
        # 이는 같은 환자로 인정
        if (sorted_df.loc[idx, "subject_id"] == sorted_df.loc[idx-1, "subject_id"]) & \
        (sorted_df.loc[idx, "hadm_id"] == sorted_df.loc[idx-1, "hadm_id"]):
            time_diff = sorted_df.loc[idx, time_col] - sorted_df.loc[idx-1, time_col]

            # time_diff가 1시간 이내라면 duplicate, 아니라면 not_duplicate
            if time_diff <= pd.Timedelta(hours=1):
                sorted_df.loc[idx, "dup"] = "duplicated"
                sorted_df.loc[idx-1, "dup"] = "duplicated"
            else:
                sorted_df.loc[idx, "dup"] = "not_duplicated"
                
        # 현재 행과 이전 행의 subject_id, hadm_id가 다르면 not_duplicate
        else:
            sorted_df.loc[idx, "dup"] = "not_duplicated"

    return sorted_df

In [5]:
intubation1_exp = intubation1.copy()
extubation1_exp = extubation1.copy()

In [6]:
start_time = time.time()

filtered_intubation = time_duplicate_event(intubation1_exp, "intubationtime")
filtered_extubation = time_duplicate_event(extubation1_exp, "extubationtime")

print(f"intubation_shape = {filtered_intubation[filtered_intubation['dup'] == 'duplicated'].shape}")
print(f"extubation_shape = {filtered_extubation[filtered_extubation['dup'] == 'duplicated'].shape}")

end_time = time.time()
wait_time = end_time - start_time

print("----")
print(f"경과 시간 : {round(wait_time, 2)} sec")


intubation_shape = (18, 7)
extubation_shape = (36, 8)
----
경과 시간 : 2.27 sec


In [13]:
filtered_intubation[filtered_intubation["dup"] == "duplicated"]

,subject_id,hadm_id,stay_id,intubationtime,itemid,patientweight,dup
18,10021927,24623461,34575919,2180-09-20 15:43:00,224385,56.6,duplicated
19,10021927,24623461,34575919,2180-09-20 16:00:00,224385,56.6,duplicated
54,10089244,29469323,33563887,2128-02-11 12:22:00,224385,90.0,duplicated
55,10089244,29469323,33563887,2128-02-11 12:30:00,224385,90.0,duplicated
63,10098215,22394571,36018186,2118-06-03 20:00:00,224385,55.4,duplicated
...,...,...,...,...,...,...,...
8085,19592126,23077014,30077165,2121-11-29 05:30:00,224385,102.4,duplicated
8229,19747328,29300235,38984649,2154-01-05 12:17:00,224385,103.9,duplicated
8230,19747328,29300235,38984649,2154-01-05 12:24:00,224385,103.9,duplicated
8415,19969918,29544887,31213286,2186-01-16 20:29:00,224385,68.0,duplicated


In [14]:
filtered_extubation[filtered_extubation["dup"] == "duplicated"]

,subject_id,hadm_id,stay_id,extubationtime,itemid,patientweight,dup
48,10023486,25262533,31361200,2151-06-08 18:00:00,227194,139.9,duplicated
49,10023486,25262533,31361200,2151-06-08 18:03:00,227194,139.9,duplicated
59,10027602,28166872,32391858,2201-11-07 13:40:00,227194,64.0,duplicated
60,10027602,28166872,32391858,2201-11-07 13:53:00,227194,64.0,duplicated
79,10038933,25129047,32166508,2148-09-11 14:00:00,227194,123.0,duplicated
...,...,...,...,...,...,...,...
22577,19807790,27131460,38192208,2148-01-23 09:45:00,225468,52.6,duplicated
22912,19952171,27208415,32980185,2177-02-02 23:27:00,227194,89.0,duplicated
22913,19952171,27208415,32980185,2177-02-02 23:35:00,227194,89.0,duplicated
23025,19997752,29452285,34531437,2128-03-02 18:08:00,227194,86.0,duplicated


In [635]:
filtered_intubation.to_csv('./Data/output/intubation1_dup.csv')
filtered_extubation.to_csv('./Data/output/extubation1_dup.csv')
filtered_intubation[filtered_intubation["dup"] == "duplicated"].to_csv('./Data/output/intubation1_dup_filter.csv')
filtered_extubation[filtered_extubation["dup"] == "duplicated"].to_csv('./Data/output/extubation1_dup_filter.csv')

### Workflow 1-2-1
* 원본 copy후, 안전하게 진행

* timedelta(time_difference)를 구하기 위해 dtype 변경

* time_col1(intubation), time_col2(extubation) 각각 Dataframe의 null 값을 없앤 후 새로운 Dataframe으로 진행

* 개별 데이터 프레임 정렬 진행

* 각 dataframe의 index list를 변수에 저장 (loc 활용 용이하도록)

* 첫 번째 행은 겹칠일이 없기에 not_duplicated 지정

* for문을 활용하여 index를 순차적 조회
    * col1_df (intubationtime이 notnull 처리된 dataframe)에 loc 활용
    * time_difference <= 1이라면, duplicate로 지정
        * 추후 확인을 위해 time_difference <= 1 인 row와 prev_row 두 개 duplicate로 지정정

In [7]:
alignment_df = pd.read_csv('./Data/alignment_df_raw_new.csv', index_col="Unnamed: 0")

In [8]:
alignment_df_exp = alignment_df.copy()

In [9]:
def time_duplicate_event_new(df, time_col1, time_col2, time_delta):
    # 원본을 copy하여 복제한 df로 진행
    origin_df = df.copy()

    # timedelta를 구하기 위해 데이터 타입 변경
    origin_df[time_col1] = pd.to_datetime(origin_df[time_col1])
    origin_df[time_col2] = pd.to_datetime(origin_df[time_col2])

    # intubation, extubation 각각 null 값을 없앤 후 새로운 df로 진행
    col1_df = origin_df[origin_df[time_col1].notnull()]
    col2_df = origin_df[origin_df[time_col2].notnull()]

    # 각각의 df의 순서를 sort
    col1_sorted_df = col1_df.sort_values(by=["subject_id", "hadm_id", time_col1])
    col2_sorted_df = col2_df.sort_values(by=["subject_id", "hadm_id", time_col2])

    # loc에 활용할 index 리스트 반환 
    col1_idx_list = col1_sorted_df.index.to_list()
    col2_idx_list = col2_sorted_df.index.to_list()
    
    
    # 첫 번째 행 (0번째 행)은 겹칠일이 없기에 not_duplicated 지정
    origin_df.loc[0, "int_dup"] = "not_duplicated"
    origin_df.loc[0, "ext_dup"] = "not_duplicated"

    # dataframe의 길이를 가지고 for문 시작.
    for idx in range(1, (len(col1_sorted_df))):

        if (col1_sorted_df.loc[col1_idx_list[idx], "subject_id"] == col1_sorted_df.loc[col1_idx_list[idx-1], "subject_id"]) & (col1_sorted_df.loc[col1_idx_list[idx], "hadm_id"] == col1_sorted_df.loc[col1_idx_list[idx-1], "hadm_id"]):
        
            time_diff = col1_sorted_df.loc[col1_idx_list[idx], time_col1] - col1_sorted_df.loc[col1_idx_list[idx-1], time_col1]
            
            origin_df.loc[col1_idx_list[idx], "int_timediff"] = time_diff
            
            if time_diff <= pd.Timedelta(minutes = time_delta):
                origin_df.loc[col1_idx_list[idx], "int_dup"] = "duplicated"
                origin_df.loc[col1_idx_list[idx-1], "int_dup"] = "duplicated"
                
            else:
                origin_df.loc[col1_idx_list[idx], "int_dup"] = "not_duplicated"
                
        else:
            origin_df.loc[col1_idx_list[idx], "int_dup"] = "not_duplicated"

    for idx in range(1, (len(col2_sorted_df))):

        if (col2_sorted_df.loc[col2_idx_list[idx], "subject_id"] == col2_sorted_df.loc[col2_idx_list[idx-1], "subject_id"]) & (col2_sorted_df.loc[col2_idx_list[idx], "hadm_id"] == col2_sorted_df.loc[col2_idx_list[idx-1], "hadm_id"]):
        
            time_diff = col2_sorted_df.loc[col2_idx_list[idx], time_col2] - col2_sorted_df.loc[col2_idx_list[idx-1], time_col2]
            
            origin_df.loc[col2_idx_list[idx], "ext_timediff"] = time_diff
            
            if time_diff <= pd.Timedelta(minutes = time_delta):
                origin_df.loc[col2_idx_list[idx], "ext_dup"] = "duplicated"
                origin_df.loc[col2_idx_list[idx-1], "ext_dup"] = "duplicated"
                
            else:
                origin_df.loc[col2_idx_list[idx], "ext_dup"] = "not_duplicated"
                
        else:
            origin_df.loc[col2_idx_list[idx], "ext_dup"] = "not_duplicated"    
    
    return origin_df

In [10]:
minutes = [10, 20, 30, 40, 50, 60]

for minute in minutes:
    filter_df = time_duplicate_event_new(alignment_df_exp, "intubationtime", "extubationtime", minute)
    total_int_row = len(filter_df[filter_df['intubationtime'].notnull()])
    total_ext_row = len(filter_df[filter_df['extubationtime'].notnull()])
    portion_dup_int = round((filter_df["int_dup"].value_counts(normalize=True).loc['duplicated']*100), 2)
    portion_dup_ext = round((filter_df["ext_dup"].value_counts(normalize=True).loc['duplicated']*100), 2)
    
    print(f"time_delta가 {minute}분 일때")
    print(f"{'-'*70}")
    print(f'intubation 중복 행 개수 : 전체 intubation 행 개수 {total_int_row}행 중에서, {filter_df["int_dup"].value_counts().loc["duplicated"]}개')
    print(f'intubation 중복 행 비율 : {portion_dup_int} %')
    print(f'extubation 중복 행 개수 : 전체 extubation 행 개수 {total_ext_row}행 중에서, {filter_df["ext_dup"].value_counts().loc["duplicated"]}개')
    print(f'extubation 중복 행 비율 : {portion_dup_ext} %')
    print("")

time_delta가 10분 일때
----------------------------------------------------------------------
intubation 중복 행 개수 : 전체 intubation 행 개수 8445행 중에서, 199개
intubation 중복 행 비율 : 2.04 %
extubation 중복 행 개수 : 전체 extubation 행 개수 7357행 중에서, 193개
extubation 중복 행 비율 : 1.98 %

time_delta가 20분 일때
----------------------------------------------------------------------
intubation 중복 행 개수 : 전체 intubation 행 개수 8445행 중에서, 303개
intubation 중복 행 비율 : 3.11 %
extubation 중복 행 개수 : 전체 extubation 행 개수 7357행 중에서, 241개
extubation 중복 행 비율 : 2.47 %

time_delta가 30분 일때
----------------------------------------------------------------------
intubation 중복 행 개수 : 전체 intubation 행 개수 8445행 중에서, 357개
intubation 중복 행 비율 : 3.66 %
extubation 중복 행 개수 : 전체 extubation 행 개수 7357행 중에서, 259개
extubation 중복 행 비율 : 2.66 %

time_delta가 40분 일때
----------------------------------------------------------------------
intubation 중복 행 개수 : 전체 intubation 행 개수 8445행 중에서, 391개
intubation 중복 행 비율 : 4.01 %
extubation 중복 행 개수 : 전체 extubation 행 개수 7357행 중에서

In [18]:
filter_df

,subject_id,hadm_id,int_stayid,admittime,intubationtime,int_itemid,int_weight,ext_stayid,extubationtime,ext_itemid,...,extubationcause,dischtime,deathtime,marker,int_ext_timediff,reintubationtime,int_dup,ext_dup,int_timediff,ext_timediff
0,10001884,26184834,37510196.0,2131-01-07 20:39:00,2131-01-11 04:30:00,224385.0,65.0,37510196.0,2131-01-12 17:40:00,227194.0,...,Planned Extubation,2131-01-20 05:15:00,2131-01-20 05:15:00,NaN,1 days 13:10:00,0 days 00:00:00,not_duplicated,not_duplicated,NaT,NaT
1,10003400,23559586,38383343.0,2137-08-04 00:07:00,2137-08-17 21:21:00,224385.0,90.5,38383343.0,2137-08-21 15:40:00,227194.0,...,Planned Extubation,2137-09-02 17:05:00,2137-09-02 17:05:00,NaN,3 days 18:19:00,0 days 00:00:00,not_duplicated,not_duplicated,NaT,NaT
2,10004401,27939719,31202136.0,2144-04-11 03:31:00,2144-04-11 05:03:00,224385.0,120.0,NaN,NaT,NaN,...,NaN,2144-04-13 17:31:00,NaN,NaN,NaN,0 days 00:00:00,not_duplicated,nan,NaT,NaT
3,10004401,29988601,32773003.0,2144-01-23 07:58:00,2144-01-27 19:00:00,224385.0,76.0,32773003.0,2144-01-30 12:30:00,227194.0,...,Planned Extubation,2144-02-06 11:45:00,NaN,NaN,2 days 17:30:00,0 days 00:00:00,not_duplicated,not_duplicated,NaT,NaT
4,10004401,29988601,32773003.0,2144-01-23 07:58:00,2144-01-30 13:00:00,224385.0,76.0,NaN,NaT,NaN,...,NaN,2144-02-06 11:45:00,NaN,NaN,NaN,0 days 00:00:00,not_duplicated,nan,2 days 18:00:00,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9742,19997367,20617667,35616526.0,2126-04-20 07:15:00,2126-05-05 18:20:00,224385.0,59.0,35616526.0,2126-05-08 17:29:00,227194.0,...,Planned Extubation,2126-05-19 14:15:00,NaN,NaN,2 days 23:09:00,NaN,not_duplicated,not_duplicated,8 days 22:06:00,6 days 00:59:00
9743,19997367,20617667,NaN,2126-04-20 07:15:00,NaT,NaN,NaN,35616526.0,2126-04-22 08:23:00,227194.0,...,Planned Extubation,2126-05-19 14:15:00,NaN,NaN,NaN,0 days 00:00:00,nan,not_duplicated,NaT,NaT
9744,19999068,21606769,30143796.0,2161-08-24 04:10:00,2161-08-25 15:34:00,224385.0,55.8,30143796.0,2161-08-28 13:35:00,227194.0,...,Planned Extubation,2161-09-02 19:00:00,NaN,NaN,2 days 22:01:00,0 days 00:00:00,not_duplicated,not_duplicated,NaT,NaT
9745,19999442,26785317,32336619.0,2148-11-19 10:00:00,2148-11-19 19:00:00,224385.0,107.5,32336619.0,2148-11-20 14:15:00,227194.0,...,Planned Extubation,2148-12-04 16:25:00,NaN,NaN,0 days 19:15:00,0 days 00:00:00,not_duplicated,not_duplicated,NaT,NaT


### 추가 검증
* admittime이 추가된 데이터로 추가적인 검증 진행

In [14]:
alignment_admittime = pd.read_csv('./Data/alignment_df_raw_with_admittime.csv', index_col="Unnamed: 0")
alignment_admittime

,subject_id,hadm_id,int_stayid,admittime,intubationtime,int_itemid,int_weight,ext_stayid,extubationtime,ext_itemid,ext_weight,extubationcause,dischtime,deathtime,marker,int_ext_timediff,reintubationtime
0,10001884,26184834,37510196.0,2131-01-07 20:39:00,2131-01-11 04:30:00,224385.0,65.0,37510196.0,2131-01-12 17:40:00,227194.0,65.0,Planned Extubation,2131-01-20 05:15:00,2131-01-20 05:15:00,NaN,1 days 13:10:00,0 days 00:00:00
1,10003400,23559586,38383343.0,2137-08-04 00:07:00,2137-08-17 21:21:00,224385.0,90.5,38383343.0,2137-08-21 15:40:00,227194.0,99.6,Planned Extubation,2137-09-02 17:05:00,2137-09-02 17:05:00,NaN,3 days 18:19:00,0 days 00:00:00
2,10004401,27939719,31202136.0,2144-04-11 03:31:00,2144-04-11 05:03:00,224385.0,120.0,NaN,NaN,NaN,NaN,NaN,2144-04-13 17:31:00,NaN,NaN,NaN,0 days 00:00:00
3,10004401,29988601,32773003.0,2144-01-23 07:58:00,2144-01-27 19:00:00,224385.0,76.0,32773003.0,2144-01-30 12:30:00,227194.0,76.0,Planned Extubation,2144-02-06 11:45:00,NaN,NaN,2 days 17:30:00,0 days 00:00:00
4,10004401,29988601,32773003.0,2144-01-23 07:58:00,2144-01-30 13:00:00,224385.0,76.0,NaN,NaN,NaN,NaN,NaN,2144-02-06 11:45:00,NaN,NaN,NaN,0 days 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9742,19997367,20617667,35616526.0,2126-04-20 07:15:00,2126-05-05 18:20:00,224385.0,59.0,35616526.0,2126-05-08 17:29:00,227194.0,59.0,Planned Extubation,2126-05-19 14:15:00,NaN,NaN,2 days 23:09:00,NaN
9743,19997367,20617667,NaN,2126-04-20 07:15:00,NaN,NaN,NaN,35616526.0,2126-04-22 08:23:00,227194.0,59.0,Planned Extubation,2126-05-19 14:15:00,NaN,NaN,NaN,0 days 00:00:00
9744,19999068,21606769,30143796.0,2161-08-24 04:10:00,2161-08-25 15:34:00,224385.0,55.8,30143796.0,2161-08-28 13:35:00,227194.0,55.8,Planned Extubation,2161-09-02 19:00:00,NaN,NaN,2 days 22:01:00,0 days 00:00:00
9745,19999442,26785317,32336619.0,2148-11-19 10:00:00,2148-11-19 19:00:00,224385.0,107.5,32336619.0,2148-11-20 14:15:00,227194.0,107.5,Planned Extubation,2148-12-04 16:25:00,NaN,NaN,0 days 19:15:00,0 days 00:00:00


In [15]:
minutes = [10, 20, 30, 40, 50, 60]

for minute in minutes:
    filter_df = time_duplicate_event_new(alignment_admittime, "intubationtime", "extubationtime", minute)
    total_int_row = len(filter_df[filter_df['intubationtime'].notnull()])
    total_ext_row = len(filter_df[filter_df['extubationtime'].notnull()])
    portion_dup_int = round((filter_df["int_dup"].value_counts(normalize=True).loc['duplicated']*100), 2)
    portion_dup_ext = round((filter_df["ext_dup"].value_counts(normalize=True).loc['duplicated']*100), 2)
    
    print(f"time_delta가 {minute}분 일때")
    print(f"{'-'*70}")
    print(f'intubation 중복 행 개수 : 전체 intubation 행 개수 {total_int_row}행 중에서, {filter_df["int_dup"].value_counts().loc["duplicated"]}개')
    print(f'intubation 중복 행 비율 : {portion_dup_int} %')
    print(f'extubation 중복 행 개수 : 전체 extubation 행 개수 {total_ext_row}행 중에서, {filter_df["ext_dup"].value_counts().loc["duplicated"]}개')
    print(f'extubation 중복 행 비율 : {portion_dup_ext} %')
    print("")

time_delta가 10분 일때
----------------------------------------------------------------------
intubation 중복 행 개수 : 전체 intubation 행 개수 8445행 중에서, 199개
intubation 중복 행 비율 : 2.04 %
extubation 중복 행 개수 : 전체 extubation 행 개수 7357행 중에서, 193개
extubation 중복 행 비율 : 1.98 %

time_delta가 20분 일때
----------------------------------------------------------------------
intubation 중복 행 개수 : 전체 intubation 행 개수 8445행 중에서, 303개
intubation 중복 행 비율 : 3.11 %
extubation 중복 행 개수 : 전체 extubation 행 개수 7357행 중에서, 241개
extubation 중복 행 비율 : 2.47 %

time_delta가 30분 일때
----------------------------------------------------------------------
intubation 중복 행 개수 : 전체 intubation 행 개수 8445행 중에서, 357개
intubation 중복 행 비율 : 3.66 %
extubation 중복 행 개수 : 전체 extubation 행 개수 7357행 중에서, 259개
extubation 중복 행 비율 : 2.66 %

time_delta가 40분 일때
----------------------------------------------------------------------
intubation 중복 행 개수 : 전체 intubation 행 개수 8445행 중에서, 391개
intubation 중복 행 비율 : 4.01 %
extubation 중복 행 개수 : 전체 extubation 행 개수 7357행 중에서

In [23]:
filter_df[filter_df["subject_id"] == 10021927]

,subject_id,hadm_id,int_stayid,admittime,intubationtime,int_itemid,int_weight,ext_stayid,extubationtime,ext_itemid,...,extubationcause,dischtime,deathtime,marker,int_ext_timediff,reintubationtime,int_dup,ext_dup,int_timediff,ext_timediff
20,10021927,24623461,34575919.0,2180-09-20 10:01:00,2180-09-20 15:43:00,224385.0,56.6,NaN,NaT,NaN,...,NaN,2180-09-27 10:32:00,2180-09-27 10:32:00,NaN,NaN,0 days 00:00:00,duplicated,nan,NaT,NaT
21,10021927,24623461,34575919.0,2180-09-20 10:01:00,2180-09-20 16:00:00,224385.0,56.6,34575919.0,2180-09-27 10:00:00,227194.0,...,Planned Extubation,2180-09-27 10:32:00,2180-09-27 10:32:00,NaN,6 days 18:00:00,0 days 00:00:00,duplicated,not_duplicated,0 days 00:17:00,NaT


In [30]:
filter_df

,subject_id,hadm_id,int_stayid,admittime,intubationtime,int_itemid,int_weight,ext_stayid,extubationtime,ext_itemid,...,extubationcause,dischtime,deathtime,marker,int_ext_timediff,reintubationtime,int_dup,ext_dup,int_timediff,ext_timediff
0,10001884,26184834,37510196.0,2131-01-07 20:39:00,2131-01-11 04:30:00,224385.0,65.0,37510196.0,2131-01-12 17:40:00,227194.0,...,Planned Extubation,2131-01-20 05:15:00,2131-01-20 05:15:00,NaN,1 days 13:10:00,0 days 00:00:00,not_duplicated,not_duplicated,NaT,NaT
1,10003400,23559586,38383343.0,2137-08-04 00:07:00,2137-08-17 21:21:00,224385.0,90.5,38383343.0,2137-08-21 15:40:00,227194.0,...,Planned Extubation,2137-09-02 17:05:00,2137-09-02 17:05:00,NaN,3 days 18:19:00,0 days 00:00:00,not_duplicated,not_duplicated,NaT,NaT
2,10004401,27939719,31202136.0,2144-04-11 03:31:00,2144-04-11 05:03:00,224385.0,120.0,NaN,NaT,NaN,...,NaN,2144-04-13 17:31:00,NaN,NaN,NaN,0 days 00:00:00,not_duplicated,nan,NaT,NaT
3,10004401,29988601,32773003.0,2144-01-23 07:58:00,2144-01-27 19:00:00,224385.0,76.0,32773003.0,2144-01-30 12:30:00,227194.0,...,Planned Extubation,2144-02-06 11:45:00,NaN,NaN,2 days 17:30:00,0 days 00:00:00,not_duplicated,not_duplicated,NaT,NaT
4,10004401,29988601,32773003.0,2144-01-23 07:58:00,2144-01-30 13:00:00,224385.0,76.0,NaN,NaT,NaN,...,NaN,2144-02-06 11:45:00,NaN,NaN,NaN,0 days 00:00:00,not_duplicated,nan,2 days 18:00:00,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9742,19997367,20617667,35616526.0,2126-04-20 07:15:00,2126-05-05 18:20:00,224385.0,59.0,35616526.0,2126-05-08 17:29:00,227194.0,...,Planned Extubation,2126-05-19 14:15:00,NaN,NaN,2 days 23:09:00,NaN,not_duplicated,not_duplicated,8 days 22:06:00,6 days 00:59:00
9743,19997367,20617667,NaN,2126-04-20 07:15:00,NaT,NaN,NaN,35616526.0,2126-04-22 08:23:00,227194.0,...,Planned Extubation,2126-05-19 14:15:00,NaN,NaN,NaN,0 days 00:00:00,nan,not_duplicated,NaT,NaT
9744,19999068,21606769,30143796.0,2161-08-24 04:10:00,2161-08-25 15:34:00,224385.0,55.8,30143796.0,2161-08-28 13:35:00,227194.0,...,Planned Extubation,2161-09-02 19:00:00,NaN,NaN,2 days 22:01:00,0 days 00:00:00,not_duplicated,not_duplicated,NaT,NaT
9745,19999442,26785317,32336619.0,2148-11-19 10:00:00,2148-11-19 19:00:00,224385.0,107.5,32336619.0,2148-11-20 14:15:00,227194.0,...,Planned Extubation,2148-12-04 16:25:00,NaN,NaN,0 days 19:15:00,0 days 00:00:00,not_duplicated,not_duplicated,NaT,NaT


In [34]:
# filter_df 전체 저장

filter_df.to_csv('./Data/output/alignment_dup_admittime_dataframe.csv', index=False)

----

## Workflow 1-2-2
* time_col1(intubation), time_col2(extubation) 데이터 프레임 각각 할당

* 개별 데이터 프레임 sort_values 진행

* time_diff를 계산하는 함수를 제작, 개별 데이터 프레임에 lambda 함수를 활용하여 적용

* 적용 뒤엔 각각의 데이터 프레임에 time_diff, dup 컬럼이 만들어졌으므로, 이를 원본 df 데이터에 loc 함수를 이용하여 할당

In [277]:
alignment_df = pd.read_csv('./Data/alignment_df_raw_new.csv', index_col="Unnamed: 0")

In [278]:
alignment_df_exp = alignment_df.copy()

In [279]:
import pandas as pd

def time_duplicate_event_new(df, time_col1, time_col2, time_delta_minute):
    # 시간 열을 datetime 형식으로 변환
    df[time_col1] = pd.to_datetime(df[time_col1])
    df[time_col2] = pd.to_datetime(df[time_col2])

    # notnull로 NaN 값이 없는 dataframe을 제작
    col1_df = df[df[time_col1].notnull()]
    col2_df = df[df[time_col2].notnull()]

    col1_sorted_df = col1_df.sort_values(by=["subject_id", "hadm_id", time_col1])
    col2_sorted_df = col2_df.sort_values(by=["subject_id", "hadm_id", time_col2])
    
    # 시간 차이를 기반으로 중복을 표시하는 함수
    def mark_duplicates(group, time_column):
        # 이전 행과의 시간 차이 계산
        group["time_diff"] = group[time_column].diff()

        result_list = ["duplicated" if td <= pd.Timedelta(minutes=time_delta_minute) else "not_duplicated" for td in group["time_diff"]]
        
        group["dup"] = result_list
        
        # 임계값과 시간 차이를 비교하여 중복 표시
        return group

    
    filter_int = col1_sorted_df.groupby(["subject_id", "hadm_id"], group_keys=False).apply(lambda x: mark_duplicates(x, time_col1))
    filter_ext = col2_sorted_df.groupby(["subject_id", "hadm_id"], group_keys=False).apply(lambda x: mark_duplicates(x, time_col2))

    col1_idx_list = filter_int.index.to_list()
    col2_idx_list = filter_ext.index.to_list()

    for idx in range(1, len(col1_idx_list)):
        # print("현재 인덱스:", idx)
        # print("filter_int 인덱스:", col1_idx_list[idx])
        if filter_int["dup"][col1_idx_list[idx]] == "duplicated":
            df.loc[col1_idx_list[idx-1], "int_dup"] = "duplicated"
            df.loc[col1_idx_list[idx], "int_dup"] = "duplicated"
            df.loc[col1_idx_list[idx], "int_timediff"] = filter_int.loc[col1_idx_list[idx], "time_diff"]
            
    for idx in range(1, len(col2_idx_list)):
        if filter_ext["dup"][col2_idx_list[idx]] == "duplicated":
            df.loc[col2_idx_list[idx-1], "ext_dup"] = "duplicated"
            df.loc[col2_idx_list[idx], "ext_dup"] = "duplicated"
            df.loc[col2_idx_list[idx], "ext_timediff"] = filter_ext.loc[col2_idx_list[idx], "time_diff"]

    return df

In [280]:
df_filter = time_duplicate_event_new(alignment_df_exp, "intubationtime", "extubationtime", 60)

In [286]:
df_filter.to_csv('./Data/output/alignment_dup_full.csv', index=False)

## 2. Replacement 함수 만들기

In [175]:
import pandas as pd

df_filter = pd.read_csv("./Data/output/alignment_dup_admittime_dataframe.csv")

In [201]:
'''
int_stayid, int_itemid, int_weight, intubationtime
ext_stayid, ext_itemid, ext_weight, extubationtime

subject_id, hadm_id가 같아야 같은 환자 + 같은 시기로 인정한다.
int_stayid와 ext_stayid가 동시에 NaN값을 가지는 경우는 없는 것으로 확인 된다.
또한 stayid, itemid, weight의 경우 int와 ext는 상호보완이 가능함.
'''
'''
단, itemid인 경우 이전에 데이터 추출 시, intubation은 1개의 값, extubation은 3개의 값을 사용함.
int_itemid 값은 1개로 통일이 가능하지만, extubation은 3가지 경우의 수가 있고, 4개의 값이 모두 다른 번호를 가지기 때문에
대체하는 것에 대해 생각이 필요할 것으로 판단됨.
'''

# def filled_nan_values(group):
#     if pd.isna(group.loc[i, 'int_stayid']):
#         group.loc[i, 'int_stayid'] = group.loc[i, 'ext_stayid']
#         group.loc[i, 'int_weight'] = group.loc[i, 'ext_weight']
#     elif pd.isna(group.loc[i, 'ext_stayid']):
#         group.loc[i, 'ext_stayid'] = group.loc[i, 'int_stayid']
#         group.loc[i, 'ext_weight'] = group.loc[i, 'int_weight']       

'''
int_time, ext_time의 NaN 값 참고사항
* 같은 행에서 int_time, ext_time 둘다 NaN 값인 경우는 존재하지 않는다.
* groupby로 묶어 처리하는 방법을 수행한다.
* group 내부에 존재하는 intubationtime, extubationtime 의 NaN 값이 각각 2개를 넘지 않는다.
* intubationtime이 NaN 값인 경우, 같은 행의 extubationtime 값이
  group 내부에 존재하는 intubationtime 값보다 더 이전이다.
'''

def filled_nan_time(group):
    nan_intubation_count = group['intubationtime'].isna().sum()
    nan_extubation_count = group['extubationtime'].isna().sum()
    # groupby 내부에서 intubationtime의 NaN 값의 개수가 1개라면
    if nan_intubation_count == 1:
        # index를 변수로 지정 (가독성)
        nan_index = group.index[group['intubationtime'].isna()][0]
        # loc를 활용하여 해당 index의 intubationtime 값을 같은 인덱스의 admittime으로 대체
        group.loc[nan_index, 'intubationtime'] = group.loc[nan_index, 'admittime']
        # 해당 그룹을 정렬 진행
        group.sort_values(by=["intubationtime", "extubationtime"], inplace=True)
        
    # 또는 groupby 내부에서 intubationtime의 NaN 값의 개수가 2개라면    
    elif nan_intubation_count == 2:
        # index를 변수에 지정
        nan_indices = group.index[group['intubationtime'].isna()]
        # NaN 값의 첫 번째 인덱스는 같은 행의 admittime으로 대체
        group.loc[nan_indices[0],'intubationtime'] = group.loc[nan_indices[0],'admittime']
        # NaN 값의 두 번째 인덱스는 이전 행의 extubationtime 값으로 대체
        group.loc[nan_indices[1],'intubationtime'] = group.loc[nan_indices[1] - 1,'extubationtime']
        # 정렬
        group.sort_values(by=["intubationtime", "extubationtime"], inplace=True)

    # groupby 내부에서 extubationtime의 NaN 값의 개수가 1개라면    
    if group['extubationtime'].isna().sum() == 1:        
        # index 변수 지정
        nan_index = group.index[group['extubationtime'].isna()][0]
        # 해당 index의 extubationtime 값을 같은 인덱스의 dischtime (퇴원시간) 으로 대체
        group.loc[nan_index, 'extubationtime'] = group.loc[nan_index,'dischtime']

    # 또는 groupby 내부에서 extubationtime의 NaN 값의 개수가 2개라면    
    elif group['extubationtime'].isna().sum() == 2:
        # index를 변수에 지정
        nan_indices = group.index[group['extubationtime'].isna()]
        # NaN 값 첫 번째 index의 extubationtime 값을 다음 행의 intubationtime 값으로 대체
        group.loc[nan_indices[0],'extubationtime'] = group.loc[nan_indices[0]+1,'intubationtime']
        # NaN 값 두 번째 index의 extubationtime 값을 같은 행의 dischtime 값으로 대체
        group.loc[nan_indices[1],'extubationtime'] = group.loc[nan_indices[1],'dischtime']

    return group

In [194]:
sorted_filled_nan = df_filter_exp.groupby(["subject_id", "hadm_id"], group_keys=False).apply(filled_nan_time)

In [207]:
sorted_filled_nan.head()

,subject_id,hadm_id,int_stayid,admittime,intubationtime,int_itemid,int_weight,ext_stayid,extubationtime,ext_itemid,...,extubationcause,dischtime,deathtime,marker,int_ext_timediff,reintubationtime,int_dup,ext_dup,int_timediff,ext_timediff
0,10001884,26184834,37510196.0,2131-01-07 20:39:00,2131-01-11 04:30:00,224385.0,65.0,37510196.0,2131-01-12 17:40:00,227194.0,...,Planned Extubation,2131-01-20 05:15:00,2131-01-20 05:15:00,NaN,1 days 13:10:00,0 days 00:00:00,not_duplicated,not_duplicated,NaN,NaN
1,10003400,23559586,38383343.0,2137-08-04 00:07:00,2137-08-17 21:21:00,224385.0,90.5,38383343.0,2137-08-21 15:40:00,227194.0,...,Planned Extubation,2137-09-02 17:05:00,2137-09-02 17:05:00,NaN,3 days 18:19:00,0 days 00:00:00,not_duplicated,not_duplicated,NaN,NaN
2,10004401,27939719,31202136.0,2144-04-11 03:31:00,2144-04-11 05:03:00,224385.0,120.0,NaN,2144-04-13 17:31:00,NaN,...,NaN,2144-04-13 17:31:00,NaN,NaN,NaN,0 days 00:00:00,not_duplicated,NaN,NaN,NaN
3,10004401,29988601,32773003.0,2144-01-23 07:58:00,2144-01-27 19:00:00,224385.0,76.0,32773003.0,2144-01-30 12:30:00,227194.0,...,Planned Extubation,2144-02-06 11:45:00,NaN,NaN,2 days 17:30:00,0 days 00:00:00,not_duplicated,not_duplicated,NaN,NaN
4,10004401,29988601,32773003.0,2144-01-23 07:58:00,2144-01-30 13:00:00,224385.0,76.0,NaN,2144-02-06 11:45:00,NaN,...,NaN,2144-02-06 11:45:00,NaN,NaN,NaN,0 days 00:00:00,not_duplicated,NaN,2 days 18:00:00,NaN


In [206]:
sorted_filled_nan.to_csv('./Data/output/sorted_filled_nan.csv', index=False)